In [1]:
import sys
import time
import json 
import pandas as pd
from app.builtin import *

sys.path.append('../app')  # for let gsdmm to work

Using TensorFlow backend.


In [2]:
models= {
    'ntm': NtmModel,
    'lda': LdaModel,
    'lftm': LftmModel,
    'gsdmm': GsdmmModel
}

# 20 NG Dataset

In [3]:
dataset = '../data/20ng.txt'
labels = '../data/20ng_labels.txt'

In [4]:
with open(labels, "r") as datafile:
    true_topics = [line.rstrip() for line in datafile if line]
true_topics[0:10]

['rec.autos',
 'comp.sys.mac.hardware',
 'comp.sys.mac.hardware',
 'comp.graphics',
 'sci.space',
 'talk.politics.guns',
 'sci.med',
 'comp.sys.ibm.pc.hardware',
 'comp.os.ms-windows.misc',
 'comp.sys.mac.hardware']

In [5]:
results = {}
for name, model in models.items():
    start = time.time()
    m = model()
    print(name)
    results[name] = m.get_corpus_predictions()
    dur = time.time() - start
    print('Done in ' + str(dur))

W0416 11:20:20.768384 4401335744 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0416 11:20:20.926007 4401335744 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ntm


W0416 11:20:21.229547 4401335744 module_wrapper.py:139] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Done in 1.9171209335327148
lda
Done in 0.5692572593688965
lftm
Done in 0.266970157623291
gsdmm
Done in 0.38367390632629395


In [6]:
df = pd.DataFrame()
df['true'] = true_topics
for name, model in models.items():
    r = results[name]
    for i in range(0,3):
        key, value = zip(*[x[i] if len(x) > i else (-1, -1) for x in r])
        df['%s_topic_%d'%(name,i)] = key
        df['%s_score_%d'%(name,i)] = value
df

,true,ntm_topic_0,ntm_score_0,ntm_topic_1,ntm_score_1,ntm_topic_2,ntm_score_2,lda_topic_0,lda_score_0,lda_topic_1,...,lftm_topic_1,lftm_score_1,lftm_topic_2,lftm_score_2,gsdmm_topic_0,gsdmm_score_0,gsdmm_topic_1,gsdmm_score_1,gsdmm_topic_2,gsdmm_score_2
0,rec.autos,11,0.227447,4,0.202119,5,0.163517,0,0.001044,1,...,3,0.220588,16,0.073529,1,0.999993,19,6.891301e-06,11,2.122357e-17
1,comp.sys.mac.hardware,14,0.249227,18,0.174287,0,0.165302,0,0.000661,1,...,8,0.146067,17,0.089888,1,1.000000,19,1.552614e-14,4,5.241309e-31
2,comp.sys.mac.hardware,7,0.333553,14,0.272124,5,0.116028,0,0.000367,1,...,10,0.160714,7,0.089286,0,0.000000,1,0.000000e+00,2,0.000000e+00
3,comp.graphics,16,0.254609,0,0.196284,2,0.137768,0,0.000881,1,...,19,0.098592,6,0.084507,1,1.000000,19,7.681433e-09,4,4.168175e-10
4,sci.space,5,0.292701,0,0.186552,10,0.141531,0,0.000629,1,...,1,0.134021,2,0.134021,19,1.000000,4,7.832201e-09,1,6.731073e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11309,sci.med,7,0.601212,8,0.236097,2,0.131402,0,0.000357,1,...,10,0.085714,3,0.080000,0,0.000000,1,0.000000e+00,2,0.000000e+00
11310,comp.sys.mac.hardware,15,0.339457,2,0.160176,19,0.155714,0,0.000862,1,...,17,0.126582,13,0.101266,1,1.000000,4,7.849075e-19,19,1.750207e-19
11311,comp.sys.ibm.pc.hardware,9,0.211234,14,0.181800,12,0.162045,0,0.000748,1,...,8,0.202532,3,0.088608,1,1.000000,19,8.755647e-13,4,6.851880e-28
11312,comp.graphics,4,0.256770,7,0.198837,16,0.172548,0,0.000558,1,...,19,0.092593,2,0.083333,4,1.000000,19,3.562953e-11,1,1.542093e-31


In [7]:
df.index.name = 'id'
df.to_csv('./out/predictions_20ng.csv')

In [8]:
for name, model in models.items():
    m = model()
    with open('./out/'+name+'_topics_20ng.json', 'w') as file:
        json.dump(m.topics(), file)

# AFP dataset

In [3]:
dataset = '../data/afp.txt'
labels = '../data/afp_labels.txt'

In [4]:
with open(labels, "r") as datafile:
    true_topics = [line.rstrip() for line in datafile if line]
true_topics[0:10]

[',,',
 '13000000,13007000 13006000 13008000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '15000000,,',
 '11000000,11014000 11001000 11002000 11006000,',
 '02000000,02003000 02001000 16001000,02003001',
 '15000000,15054000,']

In [5]:
true1, true2, true3 = zip(*[line.split(',') for line in true_topics])
true1[0:10]

('',
 '13000000',
 '11000000',
 '11000000',
 '11000000',
 '11000000',
 '15000000',
 '11000000',
 '02000000',
 '15000000')

In [ ]:
results = {}
for name, model in models.items():
    start = time.time()
    m = model()
    print(name)
    results[name] = m.get_corpus_predictions()
    dur = time.time() - start
    print('Done in ' + str(dur))

W0417 13:45:03.727105 4663692736 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ntm


W0417 13:45:03.899070 4663692736 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0417 13:45:04.289768 4663692736 module_wrapper.py:139] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

/Users/pasquale/git/ted-talk-topic-extraction/app/builtin/doc2topic/models.py:18: RuntimeWarning: invalid value encountered in true_divide
  L1normalize = (lambda x: x/L1(x))


Done in 5.400324106216431
lda
Done in 4.530890941619873
lftm


In [ ]:
df = pd.DataFrame()
df['true'] = true1
df['true2'] = true2
df['true3'] = true3
for name, model in models.items():
    r = results[name]
    for i in range(0,3):
        key, value = zip(*[x[i] if len(x) > i else (-1, -1) for x in r])
        df['%s_topic_%d'%(name,i)] = key
        df['%s_score_%d'%(name,i)] = value
df

In [ ]:
df.index.name = 'id'
df.to_csv('./out/predictions_afp.csv')

In [ ]:
for name, model in models.items():
    m = model()
    with open('./out/'+name+'_topics_afp.json', 'w') as file:
        json.dump(m.topics(), file)